In [1]:
#合并电量数据模块

import pandas as pd
import os
import re

# --- 1. 配置 ---
file_path = r"E:\A智网\电量预测数据\lightgbm模型\电量数据源文件"
output_excel_file = r"E:\A智网\月度电力需求预测报告\0_数据\湖北省行业日用电量.xlsx"
file_list = [
    "湖北行业客户用电量日时序电量监测2023-01-31.xls", "湖北行业客户用电量日时序电量监测2023-02-28.xls",
    "湖北行业客户用电量日时序电量监测2023-03-31.xls", "湖北行业客户用电量日时序电量监测2023-04-30.xls",
    "湖北行业客户用电量日时序电量监测2023-05-31.xls", "湖北行业客户用电量日时序电量监测2023-06-30.xls",
    "湖北行业客户用电量日时序电量监测2023-07-31.xls", "湖北行业客户用电量日时序电量监测2023-08-31.xls",
    "湖北行业客户用电量日时序电量监测2023-09-30.xls", "湖北行业客户用电量日时序电量监测2023-10-31.xls",
    "湖北行业客户用电量日时序电量监测2023-11-30.xls", "湖北行业客户用电量日时序电量监测2023-12-31.xls",
    "湖北行业客户用电量日时序电量监测2024-01-31.xls", "湖北行业客户用电量日时序电量监测2024-02-29.xls",
    "湖北行业客户用电量日时序电量监测2024-03-31.xls", "湖北行业客户用电量日时序电量监测2024-04-30.xls",
    "湖北行业客户用电量日时序电量监测2024-05-31.xls", "湖北行业客户用电量日时序电量监测2024-06-30.xls",
    "湖北行业客户用电量日时序电量监测2024-07-31.xls", "湖北行业客户用电量日时序电量监测2024-08-31.xls",
    "湖北行业客户用电量日时序电量监测2024-09-30.xls", "湖北行业客户用电量日时序电量监测2024-10-31.xls",
    "湖北行业客户用电量日时序电量监测2024-11-30.xls", "湖北行业客户用电量日时序电量监测2024-12-31.xls",
    "湖北行业客户用电量日时序电量监测2025-01-31.xls", "湖北行业客户用电量日时序电量监测2025-02-28.xls",
    "湖北行业客户用电量日时序电量监测2025-03-31.xls", "湖北行业客户用电量日时序电量监测2025-04-30.xls",
    "湖北行业客户用电量日时序电量监测2025-05-31.xls", "湖北行业客户用电量日时序电量监测2025-06-30.xls",
    "湖北行业客户用电量日时序电量监测2025-07-31.xls", "湖北行业客户用电量日时序电量监测2025-08-31.xls",
    "湖北行业客户用电量日时序电量监测2025-09-30.xls", "湖北行业客户用电量日时序电量监测2025-10-31.xls",
    "湖北行业客户用电量日时序电量监测2025-11-30.xls", "湖北行业客户用电量日时序电量监测2025-12-31.xls",
    "湖北行业客户用电量日时序电量监测2026-01-15.xls"
]

# 用于存放每个月处理好的【宽格式】DataFrame
all_monthly_wide_dfs = []

print("开始处理Excel (.xls) 文件 (直接宽表合并模式)...")

# --- 2. 循环处理每个文件，生成月度宽表 ---
for file_name in file_list:
    full_file_path = os.path.join(file_path, file_name)
    try:
        year_match = re.search(r'(\d{4})-\d{2}-\d{2}', file_name)
        if not year_match: continue
        year = year_match.group(1)
        
        print(f"正在读取和处理文件: {file_name}")
        
        # 步骤 2.1: 读取文件，得到多级标题的DataFrame
        df_month = pd.read_excel(full_file_path, header=[0, 1], skiprows=1, index_col=0)
        
        # 步骤 2.2: 清理第二级标题中的空格，确保能匹配到'本年'
        df_month.columns = pd.MultiIndex.from_tuples(
            [(str(col[0]).strip(), str(col[1]).strip()) for col in df_month.columns]
        )
        
        # 步骤 2.3: 直接筛选出所有'本年'的列
        df_this_year = df_month.loc[:, (slice(None), '本年')]

        # 步骤 2.4: 格式化列标题为 YYYYMMDD
        # a) 丢掉'本年'这一级标题
        date_columns = df_this_year.columns.get_level_values(0)
        # b) 将'MM/DD'格式的日期转换为'YYYYMMDD'
        formatted_columns = []
        for date_str in date_columns:
            try:
                # 使用pd.to_datetime进行稳健的日期转换和格式化
                full_date = pd.to_datetime(f"{year}-{date_str}")
                formatted_columns.append(full_date.strftime('%Y%m%d'))
            except ValueError:
                # 如果某个列名无法转换为日期（例如'行业名称'），则保持原样
                formatted_columns.append(date_str)
        
        # c) 将清理好的列名赋回DataFrame
        df_this_year.columns = formatted_columns
        
        all_monthly_wide_dfs.append(df_this_year)
        print(f"文件 {file_name} 处理成功！")
        
    except Exception as e:
        print(f"处理文件 {file_name} 时发生错误: {e}")

# --- 3. 横向合并所有月度宽表并保存 ---
if all_monthly_wide_dfs:
    print("\n所有文件处理完毕，开始合并...")
    
    # 步骤 3.1: 使用pd.concat横向拼接所有DataFrame
    # pandas会根据行索引(行业名称)自动对齐数据
    final_wide_df = pd.concat(all_monthly_wide_dfs, axis=1)
    
    # 步骤 3.2: 将行索引'行业名称'转换回普通列
    final_wide_df = final_wide_df.reset_index()
    # 可能需要重命名索引列
    final_wide_df.rename(columns={'index': '行业名称'}, inplace=True)

    # 步骤 3.3: 保存到您指定的Excel文件
    final_wide_df.to_excel(output_excel_file, index=False)
    
    print("\n处理完成！")
    print(f"数据已按您的要求格式化并保存到文件: {output_excel_file}")
    print("\n最终输出格式预览:")
    print(final_wide_df.iloc[:5, :5])
else:
    print("\n没有成功处理任何数据，请检查上面的错误信息。")

开始处理Excel (.xls) 文件 (直接宽表合并模式)...
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-01-31.xls
文件 湖北行业客户用电量日时序电量监测2023-01-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-02-28.xls
文件 湖北行业客户用电量日时序电量监测2023-02-28.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-03-31.xls
文件 湖北行业客户用电量日时序电量监测2023-03-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-04-30.xls
文件 湖北行业客户用电量日时序电量监测2023-04-30.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-05-31.xls
文件 湖北行业客户用电量日时序电量监测2023-05-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-06-30.xls
文件 湖北行业客户用电量日时序电量监测2023-06-30.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-07-31.xls
文件 湖北行业客户用电量日时序电量监测2023-07-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-08-31.xls
文件 湖北行业客户用电量日时序电量监测2023-08-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-09-30.xls
文件 湖北行业客户用电量日时序电量监测2023-09-30.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-10-31.xls
文件 湖北行业客户用电量日时序电量监测2023-10-31.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-11-30.xls
文件 湖北行业客户用电量日时序电量监测2023-11-30.xls 处理成功！
正在读取和处理文件: 湖北行业客户用电量日时序电量监测2023-12-31.xls
文件 湖北行业客户用电量日时序电量监测202